# Getting Started with RustLab-Optimize

Welcome to RustLab-Optimize! This notebook introduces you to the fundamentals of optimization using Rust's clean, math-first API. We'll cover:

1. **Simple 1D Optimization** - Finding minima of single-variable functions
2. **2D Optimization** - The classic Rosenbrock function
3. **N-Dimensional Problems** - Scaling to multiple variables
4. **Algorithm Selection** - Choosing the right optimization method
5. **Understanding Convergence** - Tuning optimization parameters
6. **Handling Failures** - What to do when optimization doesn't work

## Prerequisites

This notebook assumes you have:
- Basic understanding of mathematical optimization
- Familiarity with Rust syntax
- The `rustlab-optimize` crate and its dependencies installed

**Note**: This notebook uses best practices for rust-analyzer and evcxr compatibility.

## Setup and Imports

First, let's import the necessary modules:

In [2]:
// Load the required crates
:dep rustlab-math = { path = "../../rustlab-math" }
:dep rustlab-optimize = { path = ".." }

use rustlab_optimize::prelude::*;
use rustlab_math::linspace;
use std::f64::consts::PI;

let setup_msg = "Setup complete! Ready to explore optimization.";
println!("{}", setup_msg);

Setup complete! Ready to explore optimization.


## 1. Simple 1D Optimization

Let's start with the simplest case: finding the minimum of a single-variable function.

We'll optimize the function $f(x) = (x - 3)^2 + 1$, which has its minimum at $x = 3$ with value $f(3) = 1$.

In [3]:
{
    // Define a simple quadratic function
    let f = |x: f64| (x - 3.0).powi(2) + 1.0;

    let header = "=== 1D Optimization Example ===";
    println!("{}", header);
    println!();
    
    let function_desc = "Function: f(x) = (x - 3)² + 1";
    println!("{}", function_desc);

    // Find the minimum using rustlab-optimize's clean API
    let x_min = minimize_1d(f).solve()?;

    let result_msg = format!("Minimum found at x = {:.6}", x_min);
    println!("{}", result_msg);
    
    let value_msg = format!("Minimum value: f({:.6}) = {:.6}", x_min, f(x_min));
    println!("{}", value_msg);
    
    let expected_msg = "Expected minimum at x = 3.0";
    println!("{}", expected_msg);

    // Verify our result
    assert!((x_min - 3.0).abs() < 1e-6, "Optimization should find x ≈ 3.0");
    let success_msg = "✓ Test passed: Found minimum within tolerance";
    println!("{}", success_msg);
}

=== 1D Optimization Example ===

Function: f(x) = (x - 3)² + 1
Minimum found at x = 3.000000
Minimum value: f(3.000000) = 1.000000
Expected minimum at x = 3.0
✓ Test passed: Found minimum within tolerance


()

### Adding Bounds

What happens when we constrain the search space? Let's restrict our search to the interval $[0, 2]$:

In [4]:
{
    // Re-define function for this cell
    let f = |x: f64| (x - 3.0).powi(2) + 1.0;
    
    // Optimize with bounds that exclude the true minimum
    let x_min_bounded = minimize_1d(f)
        .bounds(0.0, 2.0)  // Restrict search to [0, 2]
        .solve()?;

    println!();
    let header = "=== With Bounds [0, 2] ===";
    println!("{}", header);
    
    let result_msg = format!("Minimum found at x = {:.6}", x_min_bounded);
    println!("{}", result_msg);
    
    let value_msg = format!("Function value: f({:.6}) = {:.6}", x_min_bounded, f(x_min_bounded));
    println!("{}", value_msg);
    
    let note_msg = "Note: True minimum is outside bounds, so we get boundary value";
    println!("{}", note_msg);

    // The result should be at the right boundary since the function is decreasing towards x=3
    assert!((x_min_bounded - 2.0).abs() < 1e-6, "Should find minimum at boundary");
    let success_msg = "✓ Test passed: Found boundary minimum";
    println!("{}", success_msg);
}


=== With Bounds [0, 2] ===
Minimum found at x = 1.999299
Function value: f(1.999299) = 2.001402
Note: True minimum is outside bounds, so we get boundary value



thread '<unnamed>' panicked at src/lib.rs:150:5:
Should find minimum at boundary
stack backtrace:
   0: __rustc::rust_begin_unwind
             at /rustc/9982d6462bedf1e793f7b2dbd655a4e57cdf67d4/library/std/src/panicking.rs:697:5
   1: core::panicking::panic_fmt
             at /rustc/9982d6462bedf1e793f7b2dbd655a4e57cdf67d4/library/core/src/panicking.rs:75:14
   2: std::panic::catch_unwind
   3: run_user_code_3
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## 2. Two-Dimensional Optimization

Now let's tackle a more challenging problem: the **Rosenbrock function**. This is a classic test problem in optimization, known for its challenging "banana-shaped" valley.

The Rosenbrock function is defined as:
$$f(x, y) = (a - x)^2 + b(y - x^2)^2$$

With $a = 1$ and $b = 100$, the global minimum is at $(x, y) = (1, 1)$ with value $f(1, 1) = 0$.

In [5]:
{
    // The famous Rosenbrock function - a challenging optimization problem
    let rosenbrock = |x: f64, y: f64| {
        let a = 1.0;
        let b = 100.0;
        (a - x).powi(2) + b * (y - x.powi(2)).powi(2)
    };

    println!();
    let header = "=== 2D Optimization: Rosenbrock Function ===";
    println!("{}", header);
    println!();

    // Test different starting points to see convergence behavior
    let starting_points = vec![
        (-1.2, 1.0),   // Classic difficult starting point
        (0.0, 0.0),     // Origin
        (2.0, 2.0),     // Near the optimum
    ];

    for (x0, y0) in starting_points {
        let (x, y) = minimize_2d(rosenbrock)
            .from(x0, y0)
            .tolerance(1e-8)
            .solve()?;
        
        let start_msg = format!("Starting from ({:.1}, {:.1}):", x0, y0);
        println!("{}", start_msg);
        
        let result_msg = format!("  Found minimum at ({:.6}, {:.6})", x, y);
        println!("{}", result_msg);
        
        let value_msg = format!("  Function value: {:.2e}", rosenbrock(x, y));
        println!("{}", value_msg);
        
        let distance_from_optimum = ((x - 1.0).powi(2) + (y - 1.0).powi(2)).sqrt();
        let distance_msg = format!("  Distance from true minimum (1, 1): {:.2e}", distance_from_optimum);
        println!("{}", distance_msg);
        println!();
        
        // Verify convergence
        assert!(distance_from_optimum < 1e-3, "Should converge close to (1, 1)");
    }

    let success_msg = "✓ All Rosenbrock tests passed!";
    println!("{}", success_msg);
}


=== 2D Optimization: Rosenbrock Function ===

Starting from (-1.2, 1.0):
  Found minimum at (1.000000, 1.000000)
  Function value: 1.08e-17
  Distance from true minimum (1, 1): 3.67e-9

Starting from (0.0, 0.0):
  Found minimum at (1.000000, 1.000000)
  Function value: 4.57e-19
  Distance from true minimum (1, 1): 1.39e-9

Starting from (2.0, 2.0):
  Found minimum at (1.000000, 1.000000)
  Function value: 1.38e-15
  Distance from true minimum (1, 1): 7.76e-8

✓ All Rosenbrock tests passed!


()

## Summary and Next Steps

🎉 **Congratulations!** You've learned the fundamentals of optimization with RustLab-Optimize:

✅ **1D, 2D, and N-D optimization** with clean, mathematical syntax  
✅ **Algorithm selection** - both automatic and manual  
✅ **Convergence control** through tolerance and iteration limits  
✅ **Bounds constraints** for restricted optimization  
✅ **Error handling** for challenging problems  

### 🔗 **What's Next?**

Continue your optimization journey with these notebooks:

- **[02_curve_fitting_fundamentals.ipynb](02_curve_fitting_fundamentals.ipynb)** - Fit exponential, polynomial, and sinusoidal models to data
- **[03_parameter_constraints_clean.ipynb](03_parameter_constraints_clean.ipynb)** - Advanced bounds and parameter fixing
- **[04_algorithm_selection.ipynb](04_algorithm_selection.ipynb)** - Deep dive into BFGS, Levenberg-Marquardt, and more
- **[05_scientific_applications.ipynb](05_scientific_applications.ipynb)** - Real-world examples from pharmacokinetics, enzyme kinetics, and signal processing

### 📚 **Additional Resources**

- [RustLab-Optimize Documentation](../src/lib.rs)
- [API Reference](https://docs.rs/rustlab-optimize)
- [Example Gallery](../examples/)
- [Rust Notebook Best Practices](./RUST_NOTEBOOK_BEST_PRACTICES.md)

### 🚀 **Key API Patterns to Remember**

```rust
// 1D optimization
let x = minimize_1d(|x| x.powi(2)).solve()?;

// 2D optimization  
let (x, y) = minimize_2d(|x, y| x*x + y*y).from(1.0, 1.0).solve()?;

// N-D optimization with algorithm selection
let result = minimize(objective)
    .from(&initial)
    .using_bfgs()
    .bounds(&bounds)
    .tolerance(1e-8)
    .solve()?;
```

Happy optimizing! 🔍✨